# 整数行列

各成分が整数の行列を整数行列と言います。

$A$を$m\times n$の整数行列とします。
整数行列の基本変形は次のように整数行列を保ちます：

* ある行（列）を−１倍する
* ２行（列）を入れ替える
* ある行（列）の整数倍を他の行（列）に加える

$Q$を$m\times n$の整数行列とします。
このとき、
$\det Q=\pm 1$である$Q$をユニモジュラ行列と言います。

また、
* $\det Q =\pm 1$と$Q^{-1}$が整数行列であることは同値です。（[証明](https://shakayami-math.hatenablog.com/entry/2019/12/27/212816)）

$A$に列基本変形を繰り返し適用するのは、ユニモジュラ行列$Q$を使って$AQ$とすることに対応します。


## 行列式因子

$A$の$k$次章行列式の最大公約数$d_k(A)$は行列式因子と呼ばれます。
$d_k(A)$はユニモジュラ行列による基本変形によって不変であることに注意しましょう。
実際、

* ある行を$-1$倍したり、行を入れ替えても、新たにできる小行列式は元の$-1$倍
* ある行の整数倍を他の行に加えた場合、新たにできる少行列式は元の少行列式と既存の少行列式の整数倍の和

## Hermite標準形

rank $m$の整数行列$A: m \times n$に対して、次のような形に変形できるユニモジュラ行列$Q$が存在します。

![hermite](figs/hermite_noraml.png)

In [1]:
import numpy as np

# 基本行列を作ります

def make_transposition_matrix(size, i, j):
    # 行列のiとj番目の行（列）を入れ替える行列を作ります
    P = np.eye(size)
    P[i, i] = 0
    P[j, j] = 0
    P[i, j] = 1
    P[j, i] = 1
    return P

M, N = 3, 4
A = np.random.randint(1, 10, (M, N))
print("### 元の行列 ###")
print(A)

print("### １行目と２行目を入れ替え ###")
P = make_transposition_matrix(M, 1, 2)
print(P @ A)

print("### １列目と２列目を入れ替え ###")
P = make_transposition_matrix(N, 1, 2)
print(A @ P)


def make_scale_matrix(size, i, s):
    # 行列のi行（列）をs倍する行列を作ります　
    T = np.eye(size)
    T[i, i] = s
    return T

print("### ２行目を２倍 ###")
T =  make_scale_matrix(M, 2, 2)
print(T @ A)


def make_add_matrix(size, i, j, s):
    # 行列のj行（列）をs倍してi行（列）に足す行列を作ります　
    E = np.eye(size)
    E[i, j] = s
    return E

print("### ２行目を３倍して１行目に追加 ###")
E =  make_add_matrix(M, 1, 2, 3)
print(E @ A)


### 元の行列 ###
[[1 4 4 5]
 [1 6 8 2]
 [6 8 8 9]]
### １行目と２行目を入れ替え ###
[[1. 4. 4. 5.]
 [6. 8. 8. 9.]
 [1. 6. 8. 2.]]
### １列目と２列目を入れ替え ###
[[1. 4. 4. 5.]
 [1. 8. 6. 2.]
 [6. 8. 8. 9.]]
### ２行目を２倍 ###
[[ 1.  4.  4.  5.]
 [ 1.  6.  8.  2.]
 [12. 16. 16. 18.]]
### ２行目を３倍して１行目に追加 ###
[[ 1.  4.  4.  5.]
 [19. 30. 32. 29.]
 [ 6.  8.  8.  9.]]


In [32]:
import numpy as np

# Hermite標準形

# M x N でrank=Rの行列
R, M, N = 3, 5, 7
np.random.seed(10)
A = np.random.randint(0, 10, (R, N))
A_left = np.repeat(A[0][None, :], M - R, axis=0)
A_left = np.random.randint(0, 10, M - R).reshape(-1, 1) * A_left
A = np.vstack([A, A_left])
A_origin = A.copy()


Q = np.eye(N)  # 右からかける正則行列

assert A.shape == (M, N)
print(A)

all_zero_count = 0
while True:
    if all_zero_count == M:
        break

    all_zero_count = 0
    for m in range(M):
        if len(np.nonzero(A[m, m+1:])[0]) == 0:
            all_zero_count += 1
            continue
        # ある行について、n以上の列の非零の絶対値最小成分を見つけます
        abs_min_idx = np.argmin(np.where(A[m, m:]==0, np.infty, np.abs(A[m, m:])))

        # 絶対値最小の要素が対角成分にくるように入れ替えます
        idx = abs_min_idx + m
        P_col = make_transposition_matrix(N, m, idx)
        A = A @ P_col
        Q = Q @ P_col

        # (m, m)番目の項を使って他の行の絶対値を小さくします
        for n in range(N):
            if n == m:
                continue
            scale = A[m, n] // A[m, m]
            E = make_add_matrix(N, m, n, -scale)
            A = A @ E
            Q = Q @ E


print(np.where(np.abs(A) < 1e-6, 0, A))

# 基本変形をまとめたやつで変形してみます
A = A_origin @ Q

print(np.where(np.abs(A) < 1e-6, 0, A))

[[ 9  4  0  1  9  0  1]
 [ 8  9  0  8  6  4  3]
 [ 0  4  6  8  1  8  4]
 [ 9  4  0  1  9  0  1]
 [27 12  0  3 27  0  3]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0.]]


Hermite標準形について、次の４つの条件は同値です

* $A$のHermite標準形が$[I O]$
* 行列式因子が$d_m(A)=1$
* 任意の整数ベクトル$b$について、$Ax=b$が整数解を持つ
* $yA$が整数ベクトルとなる任意の$y$は整数ベクトル

## Smith標準形

Hermite標準形は左下に非零要素が残っちゃいますが、更に変形すれば、より行列式因子が計算しやすいSmith標準形に変形できます。

![smith](figs/smith_normal.png)

ここで、$\alpha_2$は$\alpha_1$で割り切れる、$\alpha_3$は$\alpha_2$で割り切れる、... のような形になっています。
これを利用すると、行列式因子はこの最初の左上の成分だけを調べるだけで解決できます。